In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

### **Google Colab enviromnent**

In [2]:
try:
  from google.colab import drive
  from google.colab import drive
  IN_COLAB = True
  !git clone https://github.com/BSc-WUT/ML-API
  %cd ML-API/packetbaseml
  !pip install -e .
except:
  IN_COLAB = False

In [ ]:
!pip install awscli

In [ ]:
!make sync_data_from_s3

In [ ]:
%cd /content/GZSL/
!mkdir packetbaseml/data/interim
!python3 packetbaseml/src/data/make_dataset.py

### **Setup**

In [2]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = 'cpu'
device

'cpu'

In [3]:
%cd ..

c:\Users\bkosinski\Desktop\STUDIA\INZ\modules\ML-API\packetbaseml


### **Datasets**

In [4]:
from src.data.dataset import Dataset 
from src.data.parse_dataset import dataframe_to_dataloader
from src.data.labels import cicids2018_labels, cicids2018_train_labels, nb15v2_labels, nb15v2_train_labels

#### **CICIDS2018**

In [5]:
dataset: Dataset = Dataset(csv_file_name="./data/CICIDS2018/interim/corr_data.csv")

In [6]:
dataset.data

,Unnamed: 0,Protocol,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,...,PSH Flag Cnt,URG Flag Cnt,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Init Bwd Win Byts,Fwd Seg Size Min,Label
0,0,17.0,39.0,39.0,39.000000,0.000000,98.0,98.0,98.000000,0.000000,...,0.0,0.0,0.0,1.0,78.250,39.000000,98.000000,-1.0,8.0,0
1,1,6.0,196.0,0.0,36.000000,72.998288,1460.0,0.0,319.727273,574.435217,...,1.0,0.0,1.0,1.0,192.050,36.000000,319.727273,131.0,20.0,0
2,2,6.0,1141.0,0.0,339.636364,421.410316,1460.0,0.0,506.222222,603.973463,...,1.0,0.0,0.0,0.0,414.600,339.636364,506.222222,513.0,20.0,0
3,3,6.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000,0.000000,0.000000,-1.0,20.0,0
4,4,6.0,935.0,0.0,187.000000,418.144712,376.0,0.0,125.333333,217.083701,...,0.0,0.0,0.0,0.0,163.875,187.000000,125.333333,211.0,32.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3827066,3827066,6.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,1.0,0.0,0.0,1.0,0.000,0.000000,0.000000,0.0,24.0,14
3827067,3827067,6.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,1.0,0.0,0.0,1.0,0.000,0.000000,0.000000,0.0,24.0,14
3827068,3827068,6.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,1.0,0.0,0.0,1.0,0.000,0.000000,0.000000,0.0,24.0,14
3827069,3827069,6.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,1.0,0.0,0.0,1.0,0.000,0.000000,0.000000,0.0,24.0,14


In [30]:
labels_idx: dict = {label: i for i, label in enumerate(cicids2018_labels)}
train_labels_idx: list = [
    idx for label, idx in labels_idx.items() if label in cicids2018_train_labels
][
    :-1
]  # Not including `Label` class
test_labels_idx: list = [idx for _, idx in labels_idx.items()]

In [31]:
train_labels_idx[0] = 0 #Benign traffic should be used in training

In [32]:
label_name = "Label"

##### **Dataset quick view**

In [9]:
train_dataframe.groupby("Label")["Label",].count()

,Label
Label,
0,1078777
2,230
3,686012
4,1730
5,576191
6,41508
7,461912
8,10990
9,193360


In [35]:
train_dataframe

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
9497246,17.0,43705.0,2.0,2.0,78.0,196.0,39.0,39.0,39.000000,0.000000,...,8.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0
9107654,6.0,28614199.0,9.0,11.0,324.0,3517.0,196.0,0.0,36.000000,72.998288,...,20.0,67788.333333,82307.381226,162306.0,11909.0,9469945.0,906914.347578,10033127.0,8423754.0,0
14010168,6.0,60366096.0,11.0,9.0,3736.0,4556.0,1141.0,0.0,339.636364,421.410316,...,20.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0
8500320,6.0,40.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0
1763191,6.0,4236838.0,5.0,3.0,935.0,376.0,935.0,0.0,187.000000,418.144712,...,32.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15619886,6.0,10855.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,3
15619887,6.0,1330.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,3
15619888,6.0,1638.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,3
15619891,6.0,1480.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,3


In [11]:
test_dataframe

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6.0,141385.0,9.0,7.0,553.0,3773.0,202.0,0.0,61.444444,87.534438,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,6.0,281.0,2.0,1.0,38.0,0.0,38.0,0.0,19.000000,26.870058,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,6.0,279824.0,11.0,15.0,1086.0,10527.0,385.0,0.0,98.727273,129.392497,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,6.0,132.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,6.0,274016.0,9.0,13.0,1285.0,6141.0,517.0,0.0,142.777778,183.887722,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16232997,6.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14
16232998,6.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14
16232999,6.0,732728.0,2.0,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
16233000,6.0,22.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14


In [12]:
next(iter(train_data_loader))

[tensor([[6.0000e+00, 6.5100e+02, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          1.0000e+00],
         [6.0000e+00, 7.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          1.0000e+01],
         [6.0000e+00, 6.0066e+05, 3.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          5.0000e+00],
         ...,
         [6.0000e+00, 8.6835e+07, 2.0000e+00,  ..., 8.6800e+07, 8.6800e+07,
          5.0000e+00],
         [6.0000e+00, 1.0060e+06, 3.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          5.0000e+00],
         [6.0000e+00, 9.0890e+03, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          3.0000e+00]]),
 tensor([ 1., 10.,  5.,  3., 10.,  3.,  3.,  5.,  5.,  5.,  5.,  1.,  7.,  5.,
         10.,  7.,  3.,  7.,  3.,  7., 10., 10.,  3.,  7.,  7.,  7.,  5.,  3.,
          3.,  5.,  5.,  3.])]

In [24]:
next(iter(test_data_loader))

[tensor([[6.0000e+00, 5.2026e+06, 1.4000e+01,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [1.7000e+01, 2.9645e+05, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [1.7000e+01, 1.0120e+03, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         ...,
         [1.7000e+01, 3.0700e+02, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [1.7000e+01, 1.4464e+05, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [6.0000e+00, 8.7000e+01, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00]]),
 tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 14.,  0.,  0.,  0.,  0.,
          0.,  0.,  0., 13.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.])]

#### **NB15v2**

In [7]:
dataset: Dataset = Dataset(
    "./data/NB15v2/interim/corr_data.csv"
)

In [8]:
dataset.drop_columns(['Label'])

In [9]:
dataset.fix_data_type_to_numeric()

In [10]:
len(dataset.data.columns)

18

In [11]:
labels_idx: dict = {label: i for i, label in enumerate(nb15v2_labels)}
train_labels_idx: list = [
    idx for label, idx in labels_idx.items() if label in nb15v2_train_labels
]
test_labels_idx: list = [idx for _, idx in labels_idx.items()]

In [12]:
label_name = "Attack"

In [13]:
dataset.drop_columns(['Unnamed: 0', ""])

In [14]:
dataset.data

,PROTOCOL,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,MIN_TTL,MAX_TTL,LONGEST_FLOW_PKT,SHORTEST_FLOW_PKT,MIN_IP_PKT_LEN,MAX_IP_PKT_LEN,DST_TO_SRC_AVG_THROUGHPUT,NUM_PKTS_UP_TO_128_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,FTP_COMMAND_RET_CODE,Attack
0,6,18,27,27,27,31,32,1500,52,52,1500,81344000,18,11584,8688,0.0,2
1,6,86,27,27,27,31,32,1352,52,52,1352,633888000,110,56472,14480,0.0,2
2,6,12,27,27,27,31,32,829,52,52,829,18096000,20,10136,10136,0.0,2
3,6,50,27,27,27,31,32,1352,52,52,1352,271056000,74,30408,14480,0.0,2
4,6,18,27,27,27,31,32,1128,52,52,1128,19648000,14,14480,14480,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278666,2,0,0,0,0,254,254,40,40,0,40,0,2,0,0,0.0,5
278667,2,0,0,0,0,254,254,40,40,0,40,0,2,0,0,0.0,0
278668,2,0,0,0,0,254,254,40,40,0,40,0,2,0,0,0.0,1
278669,2,0,0,0,0,254,254,40,40,0,40,0,2,0,0,0.0,3


### **Preprare dataframe**

In [15]:
train_dataframe: pd.DataFrame = dataset.filter_data_by_labels(
    train_labels_idx, label_name
)
test_dataframe: pd.DataFrame = dataset.filter_data_by_labels(
    test_labels_idx, label_name
)

In [16]:
batch_size = 16
train_data_loader = dataframe_to_dataloader(train_dataframe, batch_size)
test_data_loader = dataframe_to_dataloader(test_dataframe, batch_size)

### **Variables**

In [16]:
word_vector_size = 100
input_dim = len(train_dataframe.iloc[0])
output_dim = len(nb15v2_labels)

In [17]:
print(f'Input dim: {input_dim}\nOutput dim: {output_dim}')

Input dim: 29
Output dim: 10


## **Word2Vec**

In [29]:
from src.models.GZSL.word2vec import LabelsEmbeddings

In [30]:
word_vector_size = 100
labels_embeddings = LabelsEmbeddings(word_vector_size, nb15v2_labels)
labels_vectors = labels_embeddings.generate_vectors(nb15v2_labels)

In [31]:
[len(v) for v in labels_vectors]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [32]:
for vector in labels_vectors:
    for word in vector:
        assert len(word) == word_vector_size


### **Map layer**
Last layer of the model should be a map between incident embeddings and labels embeddings - we want to map given input data to the most corresponding Word2Vec label vector. To do this we have to initialize weights of this layer and freeze them.

In [33]:
from src.models.GZSL.utils import map_layer_init

map_layer = map_layer_init(labels_vectors)
print(f"Size: {map_layer.size()}\nType: {map_layer.dtype}")


Size: torch.Size([10, 100])
Type: torch.float32


## **Neural Network for Network Data**

In [34]:
from src.models.GZSL.model import NetNet

In [35]:
model = NetNet(
    device=device,
    input_dim=input_dim,
    words_embeddings_dim=word_vector_size,
    output_dim=output_dim,
    labels_vectors=labels_vectors
)
model.to(device)

NetNet(
  (linear1): Linear(in_features=32, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=256, out_features=100, bias=True)
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=100, out_features=10, bias=True)
  (ReLU): LeakyReLU(negative_slope=0.01)
)

In [36]:
learning_rate = 1e-06
loss_fn = nn.NLLLoss()
model.optim = optim.Adam(model.parameters(), lr=learning_rate)
torch.autograd.set_detect_anomaly(True)


### **Training**

In [37]:
model.train_model(epochs=5, data_loader=train_data_loader, loss_fn=loss_fn)

Epoch: 0, loss: -0.688
Epoch: 1, loss: -0.562
Epoch: 2, loss: -0.812
Epoch: 3, loss: -0.688
Epoch: 4, loss: -0.75


### **Removing last layer**

In [38]:
zsl_model = nn.Sequential(*(list(model.children())[:6] + list(model.children())[7:]))
zsl_model.to(device)

Sequential(
  (0): Linear(in_features=32, out_features=512, bias=True)
  (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=256, out_features=100, bias=True)
  (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): LeakyReLU(negative_slope=0.01)
)

In [39]:
print(model)
print(zsl_model)

NetNet(
  (linear1): Linear(in_features=32, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=256, out_features=100, bias=True)
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=100, out_features=10, bias=True)
  (ReLU): LeakyReLU(negative_slope=0.01)
)
Sequential(
  (0): Linear(in_features=32, out_features=512, bias=True)
  (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=256, out_features=100, bias=True)
  (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, a

### **Evaluation**
For this step we will calculate euclidean distance and find the vector which is the closest.

In [41]:
from src.models.GZSL.model_eval import evaluate_model

evaluate_model(device, model, test_data_loader, map_layer)

[                                                  ] 1/17417 batches processed

RuntimeError: The size of tensor a (100) must match the size of tensor b (10) at non-singleton dimension 0

## **Other models**

### **DYI MLP in PyTorch**

In [35]:
from src.models.MLP.model import MLP, evaluate_model

mlp_model = MLP(device, input_dim, output_dim).to(device)
mlp_model

MLP(
  (linear1): Linear(in_features=42, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=32, bias=True)
  (linear3): Linear(in_features=32, out_features=16, bias=True)
  (ReLU): LeakyReLU(negative_slope=0.01)
)

In [36]:
learning_rate = 1e-03
loss_fn = nn.NLLLoss()
mlp_model.optim = optim.Adam(mlp_model.parameters(), lr=learning_rate)

In [ ]:
inputs, labels = next(iter(train_data_loader))
outputs = mlp_model(inputs)
(labels, outputs)

In [38]:
mlp_model.train_model(epochs=10, data_loader=train_data_loader, loss_fn=loss_fn)

Epoch: 0, loss: -0.0909
Epoch: 1, loss: -0.273
Epoch: 2, loss: -0.273
Epoch: 3, loss: 0.0


In [19]:
evaluate_model(device, mlp_model, test_data_loader)

[==================================================] 239192/239192 batches processed
Accuracy: 20.05%
Precision: 22.95%
Sensitivity: 71.12%
F1: 34.70%


In [21]:
from src.models.operations import load_model
import torch

zsl_model = torch.load(
    "./models/zsl_model.pt", map_location=torch.device("cpu")
)

In [22]:
from src.models.GZSL.model_eval import evaluate_model

evaluate_model(
    device,
    zsl_model,
    test_data_loader,
    map_layer,
)

[==================================================] 430851/430851 batches processed
Accuracy: 36.86%
Precision: 99.12
Sensitivity: 37.68
F1: 54.61


### **Scikit learn**

In [17]:
def extract_data(loader):
    features = []
    labels = []
    for batch in loader:
        x, y = batch
        features.append(x.view(x.size(0), -1))  # Flatten the features
        labels.append(y)

    return torch.cat(features).numpy(), torch.cat(labels).numpy()


In [37]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import numpy as np


def print_metrics(y_test: int, y_pred: int, method: str | None) -> None:
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:4.2f}%")

    precision = precision_score(y_test, y_pred, average=method, zero_division=np.nan)
    print(f"Precision: {precision * 100:4.2f}%")

    f1 = f1_score(y_test, y_pred, average=method)
    print(f"F1 Score: {f1 * 100:4.2f}%")

    recall = f1 * precision / (2 * precision - f1)
    print(f"Recall (Sensitivity): {recall * 100:4.2f}%")

In [19]:
X_train, y_train = extract_data(train_data_loader)
X_test, y_test = extract_data(test_data_loader)

In [20]:
import numpy as np

X_train = np.where(np.isfinite(X_train), X_train, np.nan)
X_test = np.where(np.isfinite(X_test), X_test, np.nan)

In [21]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy="mean")  

X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

#### **MLP Classifier**

In [29]:
from sklearn.neural_network import MLPClassifier


mlp = MLPClassifier(
    hidden_layer_sizes=(100,),
    max_iter=500,
    alpha=1e-6,
    solver="adam",
    verbose=10,
    random_state=1,
    learning_rate_init=1e-4,
)


mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)

Iteration 1, loss = 7.16208155
Iteration 2, loss = 4.35023882
Iteration 3, loss = 4.23011600
Iteration 4, loss = 4.23329158
Iteration 5, loss = 4.21739982
Iteration 6, loss = 4.15345882
Iteration 7, loss = 4.04989369
Iteration 8, loss = 3.92736908
Iteration 9, loss = 3.92534446
Iteration 10, loss = 4.00454492
Iteration 11, loss = 3.98734717
Iteration 12, loss = 3.97637029
Iteration 13, loss = 3.84016560
Iteration 14, loss = 3.83031146
Iteration 15, loss = 3.81538975
Iteration 16, loss = 3.78285178
Iteration 17, loss = 3.78246300
Iteration 18, loss = 3.73174890
Iteration 19, loss = 3.78216883
Iteration 20, loss = 3.64288290
Iteration 21, loss = 3.68257869
Iteration 22, loss = 3.68617644
Iteration 23, loss = 3.62506411
Iteration 24, loss = 3.65979977
Iteration 25, loss = 3.58851441
Iteration 26, loss = 3.58213375
Iteration 27, loss = 3.58588732
Iteration 28, loss = 3.45861313
Iteration 29, loss = 3.56087047
Iteration 30, loss = 3.48941982
Iteration 31, loss = 3.48237964
Iteration 32, los

In [43]:
print_metrics(y_test, y_pred, method="weighted")

Accuracy: 76.08%
Precision: 75.77%
Recall (Sensitivity): 76.08%
F1 Score: 71.65%


#### **SVM**

In [19]:
from sklearn.svm import SVC


svm_model = SVC(probability=True, verbose=True)
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

[LibSVM]

In [ ]:
y_pred = svm_model.predict(X_test)

In [ ]:
print_metrics(y_test, y_pred, method="weighted")

### **Random forest**

In [24]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [25]:
y_pred = rf_classifier.predict(X_test)

In [28]:
print_metrics(y_test, y_pred, method="macro")

Accuracy: 91.22%
Precision: 71.18%
Recall (Sensitivity): 80.00%
F1 Score: 74.33%


C:\Users\bkosinski\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
